In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE
import numpy as np

# Load the dataset
file_path = 'End_dataframe.csv'  # Update with your file path
data = pd.read_csv(file_path)

# Features and target
X = data[['Gender', 'Prior_Donation', 'Lcheek_max', 'Rcheek_max', 'nose_max', 'chin_max', 'below_nose_max', 'HRV_minmax']]
y = data['VVR_Encoded']  # Assuming 'VVR_Encoded' is the target variable

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Apply SMOTE to balance the classes in the training set
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Define the SVM model and parameter grid for Grid Search
svm = SVC(probability=True, random_state=42)
param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf', 'poly'],
    'degree': [3, 4, 5],  
    'gamma': ['scale', 'auto']
}

# Grid Search with cross-validation
grid_search = GridSearchCV(estimator=svm, param_grid=param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train_resampled, y_train_resampled)

# Best model from Grid Search
best_svm = grid_search.best_estimator_

# Make predictions on the test set
y_pred = best_svm.predict(X_test)

# Evaluate the model
print("Best Parameters from Grid Search:")
print(grid_search.best_params_)

print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# Overall cross-validation scores (on training set)
cv_scores = cross_val_score(best_svm, X_train_resampled, y_train_resampled, cv=5, scoring='accuracy')
cv_mean = np.mean(cv_scores)
cv_variance = np.var(cv_scores)

print("\nOverall Cross-Validation Scores (Training Set):")
print("Scores:", cv_scores)
print("Mean accuracy:", cv_mean)
print("Variance:", cv_variance)

# Cross-validation scores by Gender
gender_labels = {1: 'Male', 2: 'Female'}
print("\nCross-Validation Scores by Gender:")

for gender, gender_name in gender_labels.items():
    # Filter data by gender
    gender_mask = X_train_resampled['Gender'] == gender
    X_gender = X_train_resampled[gender_mask].drop(columns='Gender')  # Drop Gender column for training
    y_gender = y_train_resampled[gender_mask]

    # Perform cross-validation
    cv_scores_gender = cross_val_score(best_svm, X_gender, y_gender, cv=5, scoring='accuracy')
    print(f"\nGender: {gender_name}")
    print("Scores:", cv_scores_gender)
    print("Mean accuracy:", np.mean(cv_scores_gender))
    print("Variance:", np.var(cv_scores_gender))